# Train and Host a Boke AI Keras Model on Amazon SageMaker

Amazon SageMaker is a fully-managed service that provides developers and data scientists with the ability to build, train, and deploy machine learning (ML) models quickly. Amazon SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high-quality models. The SageMaker Python SDK makes it easy to train and deploy models in Amazon SageMaker with several different machine learning and deep learning frameworks, including TensorFlow and Keras.

In this notebook, we train and host a [Keras Sequential model](https://keras.io/getting-started/sequential-model-guide) on SageMaker. The model used for this notebook is a simple deep convolutional neural network (CNN) that was extracted from [the Keras examples](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py).

For training our model, we also demonstrate distributed training with [Horovod](https://horovod.readthedocs.io) and Pipe Mode. Amazon SageMaker's Pipe Mode streams your dataset directly to your training instances instead of being downloaded first, which translates to training jobs that start sooner, finish quicker, and need less disk space.    
<br>
<br>
<b> Instance Type and Pricing:  </b>

This sample notebook was tested using 
- the `conda_tensorflow2_p36` kernel on `ml.m5.xlarge` (notebook instance) or `Python 3 (TensorFlow 2.3 Python 3.7 CPU Optimized)` kernel `ml.m5.large` (Studio notebook) 
- with the `ml.g4dn.xlarge` processing instance type 
- and `ml.g4dn.4xlarge` training instance type 

in the `ap-northeast-1` region. The processing and training time are approximately 7 minutes and 12 minuites, respectively, using a subset of dataset only containing 8k image/text pairs on the aforementioned hardware specifications.

Price per hour depends on your region and instance type. You can reference prices on the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/).  

---
---

## Setup
First, we define a few variables that are be needed later in the example.

In [ ]:
import os
import json
import boto3

from sagemaker import Session
from sagemaker import get_execution_role

sagemaker_session = Session()
default_bucket = sagemaker_session.default_bucket()
role = get_execution_role()

## Bokete Bokekan dataset

Bokete is one of the most popular commedy web site. [Bokekan dataset](../README.md) consists of 1M+ images/text pairs to 4 different classes. Here are the classes in the dataset, as well as 1 random image/text pair:

| class | boke (image/text pair) | number of stars |
| ---- | ----: | ---- |
| blue | 98,736 | 0 |
| yellow | 955,901 | 1 - 100 |
| green | 37,342 | 101 - 1000 |
| red | 8,183 | 1001 - 10000 |
| sp | 380 | 10001+ |
| Total | 1,100,542 | boke |

<img src="https://d1.awsstatic.com/Developer%20Marketing/jp/magazine/2020/img_bokete_01.a9c39e30940cf3c6fb5ba795b9202a57a843da05.jpg" alt="bokete" width="320"/>
<center>(Photo by IHA Central Office, licensed under the Creative Commons Attribution License 2.0)<center/>

Here, you will copy the Bokekan dataset to your SageMaker default bucket. 

In [ ]:
bokekan_bucket_name = 'bokekan'
prefix = '21-6-1'

In [ ]:
bokekan = ['blue_000', 'yellow_000', 'yellow_001', 'yellow_002', 'yellow_003','yellow_004', 'yellow_005', 'yellow_006','yellow_007', 'yellow_008', 'yellow_009', 'green_000', 'red_000', 'sp_000']
data_path = '../../data'

In [ ]:
%%time 
s3 = boto3.client('s3')

for boke_class in bokekan:
    key = boke_class + '.zip'
    print(bokekan_bucket_name, '=>', os.path.join('<my-default-bucket>', bokekan_bucket_name, prefix, key))
    
    s3.copy_object(
        CopySource=os.path.join(bokekan_bucket_name, prefix, key), 
        Bucket=default_bucket, 
        Key=os.path.join(bokekan_bucket_name, prefix, key), 
    )

### Prepare the dataset for training

To use the bokekan dataset, we first download it and convert it to numpy array. This step takes around 7 minutes.

In [ ]:
# !pygmentize script/keras/processing/preprocessing.py

In [ ]:
from sagemaker.tensorflow import TensorFlowProcessor

tags = [{'Key': 'Project', 'Value': 'bokete'}]

tf_processor = TensorFlowProcessor(
    framework_version="2.3",
    role=role,
    instance_type="ml.g4dn.xlarge", 
    instance_count=1,
    volume_size_in_gb=100, 
    base_job_name='bokete-tf-keras-preprocessing', 
    py_version='py37', 
    tags=tags
)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput, FeatureStoreOutput

extract_destination = "s3://{}/{}/{}/{}".format(default_bucket, bokekan_bucket_name, prefix, "contents")

tf_processor.run(
    code="preprocessing.py",
    source_dir="script/keras/processing", 
    inputs=[
        ProcessingInput(
            input_name='bokekan',
            source="s3://{}/{}/{}".format(default_bucket, bokekan_bucket_name, prefix), 
            destination="/opt/ml/processing/input", 
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="bokekan_contents", 
            source="/opt/ml/processing/output/bokekan/content", 
#             destination=extract_destination, 
            s3_upload_mode="Continuous", 
        ),
        ProcessingOutput( # params, word_id, id_word
            output_name="bokekan_metadata", 
            source="/opt/ml/processing/output/bokekan/metadata", 
#             feature_store_output=FeatureStoreOutput(FeatureGroupName='')
        ),
        ProcessingOutput( # X1, X2, y
            output_name="training", 
            source="/opt/ml/processing/output/training", 
        ),
    ],
    arguments=["--bokekan", bokekan[-2]]
)

The above processing job will upload the training data to Amazon S3. Let's retrieve the path for the next step. 

In [ ]:
preprocessing_job_description = tf_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "training":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "bokekan_metadata":
        preprocessed_bokekan_metadata = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "bokekan_contents":
        preprocessed_bokekan_contents = output["S3Output"]["S3Uri"]

In [ ]:
# retrieve parameters
!aws s3 cp {preprocessed_bokekan_metadata}/param.json ./keras/

with open('./keras/param.json') as json_file:
    param = json.load(json_file)
    print(param)

max_word = param['max_word']
vocabulary_size = param['vocabulary_size']

----

## Train the model

In this tutorial, we train a deep CNN+LSTM to learn a image captioning task with the bokekan dataset. 

### Configure metrics

In addition to running the training job, Amazon SageMaker can retrieve training metrics directly from the logs and send them to CloudWatch metrics. Here, we define metrics we would like to observe:

In [ ]:
metric_definitions = [
    {'Name': 'train:loss', 'Regex': '.*loss: ([0-9\\.]+) - accuracy: [0-9\\.]+.*'},
    {'Name': 'train:accuracy', 'Regex': '.*loss: [0-9\\.]+ - accuracy: ([0-9\\.]+).*'},
    {'Name': 'sec/steps', 'Regex': '.* - \d+s (\d+)[mu]s/step - loss: [0-9\\.]+ - accuracy: [0-9\\.]+'} 
]

### Run a baseline training job on SageMaker

The SageMaker Python SDK's `sagemaker.tensorflow.TensorFlow` estimator class makes it easy for us to interact with SageMaker. Here, we create one to configure a training job. Some parameters worth noting:

* `entry_point`: our training script (adapted from [this Keras example](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)).
* `metric_definitions`: the metrics (defined above) that we want sent to CloudWatch.
* `train_instance_count`: the number of training instances. Here, we set it to 1 for our baseline training job.

For more details about the TensorFlow estimator class, see the [API documentation](https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html).

In [ ]:
from sagemaker.tensorflow import TensorFlow

hyperparameters = {'epochs': 10, 'batch-size': 256, 'max_word': max_word, 'vocabulary_size': vocabulary_size}
tags = [{'Key': 'Project', 'Value': 'bokete'}]

estimator = TensorFlow(entry_point='keras_bokete.py',
                       source_dir='./script/keras/training', 
                       metric_definitions=metric_definitions,
                       hyperparameters=hyperparameters,
                       role=role,
                       framework_version='2.3',
                       py_version='py37',
                       instance_count=1,
                       instance_type='ml.g4dn.xlarge',
#                        instance_type='ml.g4dn.4xlarge',
#                        instance_type='ml.g5.4xlarge', 
                       base_job_name='bokete-tf-keras-training',
                       tags=tags)

estimator.fit(preprocessed_training_data)

Once we have our estimator, we call `fit()` to start the SageMaker training job and pass the inputs that we uploaded to Amazon S3 earlier. We pass the inputs as a dictionary to define different data channels for training.

### View the job training metrics

We can now view the metrics from the training job directly in the SageMaker console.  

Log into the [SageMaker console](https://console.aws.amazon.com/sagemaker/home), choose the latest training job, and scroll down to the monitor section. Alternatively, the code below uses the region and training job name to generate a URL to CloudWatch metrics.

Using CloudWatch metrics, you can change the period and configure the statistics.

In [ ]:
from urllib import parse

from IPython.core.display import Markdown

region = sagemaker_session.boto_region_name
cw_url = parse.urlunparse((
    'https',
    '{}.console.aws.amazon.com'.format(region),
    '/cloudwatch/home',
    '',
    'region={}'.format(region),
    'metricsV2:namespace=/aws/sagemaker/TrainingJobs;dimensions=TrainingJobName;search={}'.format(estimator.latest_training_job.name),
))

display(Markdown('CloudWatch metrics: [link]({}). After you choose a metric, '
                 'change the period to 1 Minute (Graphed Metrics -> Period).'.format(cw_url)))

## Deploy the trained model

After we train our model, we can deploy it to a SageMaker Endpoint, which serves prediction requests in real-time. To do so, we simply call `deploy()` on our estimator, passing in the desired number of instances and instance type for the endpoint.

Because we're using TensorFlow Serving for deployment, our training script saves the model in TensorFlow's SavedModel format.  

We don't need accelerated computing power for inference, so let's switch over to a <b>ml.m4.xlarge</b> instance type. 

For more information about deploying Keras and TensorFlow models in SageMaker, see [this blog post](https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker).

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')

### Invoke the endpoint

To verify the that the endpoint is in service, we generate some random data in the correct shape and get a prediction.

In [ ]:
!pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import reshape, stack, convert_to_tensor

In [ ]:
dummy_image = np.random.randn(1,4096)
dummy_text = np.random.randn(1,max_word)
payload = {"inputs": {"image": dummy_image.tolist(), "text": dummy_text.tolist()}}
predictor.predict(payload)['outputs'][0][:10]

Now let's use the test dataset for predictions. We will setup for inference preprocessing. 

In [ ]:
from tensorflow.keras.models              import Model
from tensorflow.keras.applications.vgg16  import VGG16 

VGG = VGG16(weights='imagenet')
VGG._layers.pop()
VGG = Model(inputs=VGG.inputs, outputs=VGG.layers[-1].output)

In [ ]:
# load dictionary
id_word = pd.read_csv(os.path.join(preprocessed_bokekan_metadata, 'id_word_bokete_data.csv'), index_col=0).to_dict()['1']

In [ ]:
# sample the first word from a distribution
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float32")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

You can find your favorite image from https://bokete.jp/boke/select, then copy&paste the image url. 

In [ ]:
image_url = 'https://d2dcan0armyq93.cloudfront.net/photo/odai/600/2182ab815bdb2abae3b67e723861ac08_600.jpg'
# image_url = 'https://d2dcan0armyq93.cloudfront.net/photo/odai/600/0235436e0d5d54c882f485e367f68fe6_600.jpg'
# image_url = 'https://d2dcan0armyq93.cloudfront.net/photo/odai/600/db6e0eb2dd26330e1884d0812d78e783_600.jpg'

!wget {image_url} ./
image_path = image_url.split('/')[-1]

In [ ]:
# # Please specify the `image_id` here.
# image_id = 256
# boke_class = bokekan[0]
# data = pd.read_csv(os.path.join(preprocessed_bokekan_metadata, 'boke.csv'))
# image_path = os.path.join(data_path, boke_class, data.odai_photo_url[image_id][1:])
# print(image_path)

# Preprocess test image array
img = load_img(image_path, target_size=(224, 224))
x = np.array(img)/255

x = VGG(reshape(x, [1, 224, 224, 3]))
X_test_image = np.array(x)

# Prepare empty text array
x2 = np.zeros([1, max_word])
x2[0, -1] = 1. # set {1: 'Beginning Of Sentence (BOS)'} at the beginning 
X_test_text = np.array(x2)

With the data loaded, we can use it for predictions:

In [ ]:
%%time

max_length = max_word
for i in range(max_length):
    payload = {"inputs": {"image": X_test_image.tolist(), "text": X_test_text.tolist()}}
    outputs = np.array(predictor.predict(payload)['outputs'])

    # sampling the initial word
    if i == 0:
        X_test_text = np.append(X_test_text, sample(outputs[0], 1.0))[1:].reshape(1,-1)

    elif outputs.argmax() == 0:
        break

    else:
        X_test_text = np.append(X_test_text, outputs.argmax())[1:].reshape(1,-1)

With the predictions, we calculate our model accuracy and create a confusion matrix.

In [ ]:
from operator import itemgetter
boke = ''.join(itemgetter(*X_test_text[0].tolist())(id_word)).split('BOS')[-1]

plt.imshow(img)
print(boke) 

Aided by the colors of the heatmap, we can use this confusion matrix to understand how well the model performed for each label.print

## Cleanup

To avoid incurring extra charges to your AWS account, let's delete the endpoint we created:

In [ ]:
predictor.delete_endpoint()

----

----

## TEST
send
- 'image/jpeg' base64 image
- 'url text' to download from S3/CF
- 'relative path text' 



[`input_handler`]  -> [`model`] x $N$-times -> [`output_handler`] 

In [ ]:
%%time
# create a test endpoint

from sagemaker.tensorflow.model import TensorFlowModel
predictor.delete_endpoint() # previous one

print('create another endpoint...')
model = TensorFlowModel(
    estimator.model_data, 
    role=role, 
    entry_point='inference.py', 
    source_dir='./script/keras/', 
    framework_version='2.3'
)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.g4dn.xlarge')


In [ ]:
import numpy as np

dummy_image = np.random.randn(1,4096)
dummy_text = np.random.randn(1,48)

payload = {"inputs": {"image": dummy_image.tolist(), "text": dummy_text.tolist()}}

predictor.predict(payload)['outputs']

## TODO
- Use SageMaker Feature Store (`id_word` dict)
- run multiple inference rounds in a single endpoint API call

In [ ]:
import sagemaker
sagemaker.__version__